<a href="https://colab.research.google.com/github/Flumenlucidum/ML-DL/blob/master/Seq2seq_RNN_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
##https://github.com/bentrevett/pytorch-seq2seq/blob/main/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [ ]:
#random seeds for deterministic results

seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
dataset = datasets.load_dataset("bentrevett/multi30k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [ ]:
train_data, valid_data,test_data=(dataset['train'], dataset['validation'], dataset['test'])

In [ ]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [ ]:
#Tokenizer with spacy 'good morning' -> ['good', 'morning']
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 18.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
en_nlp = spacy.load('en_core_web_sm')
de_nlp = spacy.load('de_core_news_sm')

In [ ]:
string = 'Please bring me a cup of tea'

['0']+[token.text for token in en_nlp.tokenizer(string)]+['1']

['0', 'Please', 'bring', 'me', 'a', 'cup', 'of', 'tea', '1']

In [ ]:
def tokenize_example(example, en_nlp, de_nlp, max_len, lower, sos_token, eos_token):
  en_tokens = [token.text for token in en_nlp(example['en'])][:max_len]
  de_tokens = [token.text for token in de_nlp(example['de'])][:max_len]
  if lower:
    en_tokens = [token.lower() for token in en_tokens]
    de_tokens = [token.lower() for token in de_tokens]
  en_tokens = [sos_token] + en_tokens + [eos_token]
  de_tokens = [sos_token] + de_tokens + [eos_token]
  return {'en_tokens':en_tokens, 'de_tokens': de_tokens}

In [ ]:
max_len = 1000
lower = True
sos_token = '<sos>'
eos_token = '<eos>'
fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_len": max_len,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}
train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs) #tokenize example returns dict
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [ ]:
#min freq설정-> 이 이하일 경우 training set에서 index설정하지 않음
#그렇게 해서 test나 validation에서 처음본단어있는 문장을 학습 잘할수있게
#special token unk sos eos pad
min_freq = 2
unk_token = '<unk>'
pad_token = '<pad>'
special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(train_data['en_tokens'],min_freq=min_freq, specials=special_tokens)
de_vocab = torchtext.vocab.build_vocab_from_iterator(train_data['de_tokens'],min_freq=min_freq, specials=special_tokens)

In [ ]:
en_vocab.get_itos()[:10] #int to string

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [ ]:
de_vocab.get_itos()[:9]  #order of frequency

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine']

In [ ]:
en_vocab['the']
en_vocab.get_stoi()['pay']

2350

In [ ]:
print(len(en_vocab), len(de_vocab))
'the' in en_vocab

5893 7853


True

In [ ]:
#en_vocab['The'] returns error

In [ ]:
#torchtext vocab class does not return zero for the unseen word so
#you need to manually set which value to return
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]
unk_index = en_vocab[unk_token]
pad_index = de_vocab[pad_token]

In [ ]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)
en_vocab['The']
en_vocab.get_itos()[0]

'<unk>'

In [ ]:
tokens = ['i', 'hate', 'flying', 'bugs']
en_vocab.lookup_indices(tokens)

[956, 4105, 536, 4716]

In [ ]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'hate', 'flying', 'bugs']

In [ ]:
def numericalize_example(example, en_vocab, de_vocab):
  en_ids = en_vocab.lookup_indices(example['en_tokens'])
  de_ids = de_vocab.lookup_indices(example['de_tokens'])
  return {'en_ids':en_ids, 'de_ids':de_ids}

In [ ]:
fn_kwargs = {'en_vocab':en_vocab, 'de_vocab': de_vocab}
train_data = train_data.map(numericalize_example,fn_kwargs = fn_kwargs)
valid_data = valid_data.map(numericalize_example,fn_kwargs = fn_kwargs)
test_data = test_data.map(numericalize_example,fn_kwargs = fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [ ]:
en_vocab.lookup_tokens(train_data['en_ids'][0])

['<sos>',
 'two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.',
 '<eos>']

In [ ]:
data_type = 'torch'
format_columns = ['en_ids','de_ids']
#Dataset class has with_format method: converts features(columns) to designated types
train_data = train_data.with_format(
    type = data_type,
    columns = format_columns,
    output_all_columns=True #O.W.remove features other than columns
)
valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)
test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [ ]:
type(train_data[0]['en_ids'])
type(train_data)

datasets.arrow_dataset.Dataset

In [ ]:
#Final step: Create Data Loaders

def get_collate_fn(pad_index):
  def collate_fn(batch):
    batch_en_ids = [example['en_ids'] for example in batch]
    batch_de_ids = [example['de_ids'] for example in batch]
    #Set maximum length of the sentences among the batch   ---- question: not all data?
    batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
    batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
    batch = {
        'en_ids' : batch_en_ids,
        'de_ids' : batch_de_ids
    }
    return batch

  return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
  collate_fn = get_collate_fn(pad_index)
  data_loader = torch.utils.data.DataLoader(
      dataset=dataset,
      batch_size=batch_size,
      collate_fn=collate_fn,
      shuffle=shuffle
  )
  return data_loader

In [ ]:
#to reduce time largest batch possible
batch_size = 128
train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

AttributeError: 'Dataset' object has no attribute 'to'

In [ ]:
#Building model in 3 parts encoder, decoder, seq2seq
class Encoder(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
    self.dropout = nn.Dropout(dropout)

  def forward(self,src):
    #src = [src len, batch_size]
    embedded = self.dropout(self.embedding(src))
    # print(embedded.shape)  #src len, batchsize, embedding dim
    outputs, (hidden, cell) = self.rnn(embedded)
    # print(outputs.shape) #src length, batch size, hidden dim * n directions
    # print(hidden.shape)  #n layers * n directions, batch size, hidden dim
    # print(cell.shape)    #cell = [n layers * n directions, batch size, hidden dim
    # outputs are always from the top hidden layer
    return hidden, cell

In [ ]:
class Decoder(nn.Module): #nn.Module 없어도 error 안떠
  def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.output_dim = output_dim
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.embedding = nn.Embedding(output_dim, embedding_dim)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
    self.fc_out = nn.Linear(hidden_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, cell):
    # input = [batch size]
    # hidden = [n layers * n directions, batch size, hidden dim]
    # cell = [n layers * n directions, batch size, hidden dim]
    # n directions in the decoder will both always be 1, therefore:
    # hidden = [n layers, batch size, hidden dim]
    # context = [n layers, batch size, hidden dim]
    input = input.unsqueeze(0)
    # input = [1, batch size]
    embedded = self.dropout(self.embedding(input))
    # embedded = [1, batch size, embedding dim]
    output, (hidden, cell) = self.rnn(embedded, (hidden, cell)) #https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    # output = [seq length, batch size, hidden dim * n directions]
    # hidden = [n layers * n directions, batch size, hidden dim]
    # cell = [n layers * n directions, batch size, hidden dim]
    # seq length and n directions will always be 1 in this decoder, therefore:
    # output = [1, batch size, hidden dim]
    # hidden = [n layers, batch size, hidden dim]
    # cell = [n layers, batch size, hidden dim]
    prediction = self.fc_out(output.squeeze(0))
    # prediction = [batch size, output dim]
    return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        #hidden_dim and n_layers must be same for encoder and decoder
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        )
        assert(
            encoder.n_layers == decoder.n_layers
        )

    def forward(self, src, trg, teacher_forcing_ratio):
        #src [src_length, batchsize]
        #trg [trg_length, batchsize]
        #probability to use teacher forcing : 0.75 = using GT input 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)

        #last hidden state of the encoder is used as initial hidden state of the decoder
        hidden,cell = self.encoder(src)
        input = trg[0,:] # 128개 문장의 <SOS>부터 다 같이 들어가  '개' only
        #이렇게 하는 이유는 teacher forcing

        for t in range(1, trg_length):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs

In [ ]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256   #7853 dimension of Deutsch vocab into 256 dim
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim, n_layers, encoder_dropout)
decoder = Decoder(output_dim, decoder_embedding_dim, hidden_dim, n_layers, decoder_dropout)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
#Initializing weight : apply function for weight initialization in all modules
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data,-0.08,0.08)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
#Count the number of params in models
def count_param(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_param(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = pad_index)

In [ ]:
!pip install torchinfo
from torchinfo import summary

In [ ]:
summary(model)

Layer (type:depth-idx)                   Param #
Seq2Seq                                  --
├─Encoder: 1-1                           --
│    └─Embedding: 2-1                    2,010,368
│    └─LSTM: 2-2                         3,678,208
│    └─Dropout: 2-3                      --
├─Decoder: 1-2                           --
│    └─Embedding: 2-4                    1,508,608
│    └─LSTM: 2-5                         3,678,208
│    └─Linear: 2-6                       3,023,109
│    └─Dropout: 2-7                      --
Total params: 13,898,501
Trainable params: 13,898,501
Non-trainable params: 0

In [ ]:
#Dimensionality check
for i, batch in enumerate(test_data_loader):
    test_src_batch = batch['de_ids']
    test_trg_batch = batch['en_ids']
    break

In [ ]:
print(test_src_batch.shape)
print(test_trg_batch.shape)
# 단 개  길이 31의 문장 128개
print(test_src_batch[:,111]) #1 is pad token

torch.Size([29, 128])
torch.Size([31, 128])
tensor([   2,    5,   25,   31,   11,    6, 1838,   80,   12,    6, 2793,   87,
          35,    5, 5307,   25,    5,  148,   68,    4,    3,    1,    1,    1,
           1,    1,    1,    1,    1])


In [ ]:
print(len(de_vocab))
print(len(en_vocab))
embedded = nn.Dropout(0.5)(nn.Embedding(len(de_vocab),256)(test_src_batch)) #test case의 1190이 숫자에 해당하는 256dim vector
# 개단차개념으로는 128 31 7853 - > 128 31 256 dropout does not effect dim
print(embedded.shape)  #단개차  #dim 0의 숫자들에 대해 embedding하는게 default로 보임

7853
5893
torch.Size([29, 128, 256])


In [ ]:
embedding_dim=256
hidden_dim=512
n_layers =2
dropout=0.5
#31 128 256 단개차 들어감
out, (hidden, cell) = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)(embedded)
print(f'out : {out.shape}') #dim 256 -> 512
print(f'hidden : {hidden.shape}') #2
print(f'cell : {cell.shape}') #encoder out은 안쓰임
#여기서 hidden, cell 은 마지막 hidden cell state

out : torch.Size([29, 128, 512])
hidden : torch.Size([2, 128, 512])
cell : torch.Size([2, 128, 512])


In [ ]:
enc_hidden, enc_cell = encoder(test_src_batch)
print(enc_hidden.shape) #context vector  src문장을 반영한 512차원 vector
print(enc_cell.shape)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
dec_input = test_trg_batch[0,:].unsqueeze(0) #target batch 각문장 첫단어
print(dec_input.shape)

In [ ]:
dec_embedded = nn.Dropout(0.5)(nn.Embedding(len(en_vocab),256)(dec_input)) # 1 128에서 각 숫자를 256dim vector로
# 개단차개념으로는 128, 1, 5893 - > 128 1 256 dropout does not effect dim
print(dec_embedded.shape)  #단개차

In [ ]:
embedding_dim=256
hidden_dim=512
n_layers =2
dropout =0.5
#1 128 256 단개차 들어감
dec_out, (dec_hidden, dec_cell) = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)(dec_embedded,(enc_hidden, enc_cell))
# dec_embedded 1 128 256 -> 1 128 512   2 128 512 2 128 512 각 칸이 LSTM layer
print(f'dec_out : {dec_out.shape}') #dim 256 -> 512
print(f'dec_hidden : {dec_hidden.shape}') #2
print(f'dec_cell : {dec_cell.shape}') #encoder out은 안쓰임
#dec hidden, dec cell은 다음 단어 들어올때 hidden cell

In [ ]:
#이제 결과값 1 128 512 여기서 예측값 도출 첫 차원 없애고 128개 문장 첫단어 각각 512차원벡터
dec_pred = nn.Linear(hidden_dim, len(en_vocab))(dec_out.squeeze(0))  #512->5893
print(dec_pred.shape)

In [ ]:
# outputs = torch.zeros(trg_length, batch_size, trg_vocab_size)
outputs=torch.zeros(test_trg_batch.shape[0], test_trg_batch.shape[1], len(en_vocab))
print(outputs.shape) # 31 128 5893 단개차
print(outputs[0].shape) # 128 5893 개차  첫단어 여기에 예측치저장
outputs[0]=dec_pred

In [ ]:
#teacher_force = random.random() < teacher_forcing_ratio => training 때만
#get the highest predicted token
top1 = dec_pred.argmax(1) #128 5893에서 각문장의 5893개숫자중 가장 큰 놈 인덱스, 즉 128개
dec_input_new= top1 #128개  #sos token과 context vector(hidden, cell) 입력받아 예측된 next token ->다음 입력으로
# input = test_trg_batch[0] if teacher_force else top1
print(dec_input_new)

In [ ]:
#전체 과정 반복
dec_input_new = dec_input_new.unsqueeze(dim=0)  # 1 128 두번째입력->두번째위치 예측
dec_embedded = nn.Dropout(0.5)(nn.Embedding(len(en_vocab),256)(dec_input_new)) # 1 128 256  #각 숫자 256dim vector
dec_out, (dec_hidden, dec_cell) = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)(dec_embedded,(dec_hidden, dec_cell))
dec_pred = nn.Linear(hidden_dim, len(en_vocab))(dec_out.squeeze(0)) # 128 5893
outputs[1]=dec_pred #index늘려나가
top1=dec_pred.argmax(1) # 128
dec_input_new=top1 #각문장 두번째 위치에 올 단어 예측
print(dec_input_new)

## Training model

In [ ]:
def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch['de_ids'].to(device) #29 128
        trg = batch['en_ids'].to(device) #31 128
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio) # 31 128 5893
        output_dim = output.shape[-1]
        output = output[1:].view(-1,output_dim) #30 128 5893
        trg = trg[1:].view(-1) #30 128
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
##Evaluation loop
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch['de_ids'].to(device) #29 128
            trg = batch['en_ids'].to(device) #31 128
            #없는 부분 체크
            # optimizer.zero_grad()
            output = model(src, trg, 0) # 31 128 5893 #always teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1,output_dim) #30 128 5893
            trg = trg[1:].view(-1) #30 128
            loss = criterion(output, trg)
            # loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters, clip)
            # optimizer.step()
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
EPOCH=10
CLIP=1.0
teacher_forcing_ratio=0.5

best_valid_loss=float('inf')

for epoch in tqdm.tqdm(range(EPOCH)):
    train_loss=train_fn(model,
                        train_data_loader,
                        optimizer,
                        criterion,
                        CLIP,
                        teacher_forcing_ratio,
                        device)
    valid_loss=evaluate_fn(model,
                        valid_data_loader,
                        criterion,
                        device)
    if valid_loss < best_valid_loss:
        best_valid_loss =valid_loss
        torch.save(model.state_dict(),'tut1-model.pt')
    print(f"\tTrain Loss : {train_loss:7.3f} | Train PPL : {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss : {valid_loss:7.3f} | Valid PPL : {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [00:49<07:29, 49.92s/it]

	Train Loss :   5.038 | Train PPL : 154.203
	Valid Loss :   4.954 | Valid PPL : 141.803


 20%|██        | 2/10 [01:38<06:31, 48.93s/it]

	Train Loss :   4.446 | Train PPL :  85.296
	Valid Loss :   4.737 | Valid PPL : 114.074


 30%|███       | 3/10 [02:32<05:59, 51.34s/it]

...

	Train Loss :   3.289 | Train PPL :  26.825
	Valid Loss :   4.000 | Valid PPL :  54.578


 90%|█████████ | 9/10 [07:43<00:50, 50.57s/it]

	Train Loss :   3.173 | Train PPL :  23.887
	Valid Loss :   3.898 | Valid PPL :  49.323


100%|██████████| 10/10 [08:33<00:00, 51.38s/it]

	Train Loss :   3.040 | Train PPL :  20.905
	Valid Loss :   3.814 | Valid PPL :  45.315


In [ ]:
#testing model performance
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate_fn(model, test_data_loader, criterion, device)
print(f'Test Loss : {test_loss:.3f} | Test PPL : {np.exp(test_loss):7.3f}')

Test Loss : 3.808 | Test PPL :  45.052


In [ ]:
#BLEU score we need translate_sentence function
##PPEN Note check